# The Double Slit Experiment - FEniCSx
Author: Roberto Carlos Gómez Araque

This notebook demonstrates the simulation of the classic double slit experiment using the finite element method with FEniCSx. We solve the Helmholtz equation to model acoustic wave propagation through a domain with two narrow slits.

![Module of the pressure field](result_p_modulus.png)

## Problem Description

The double slit experiment involves:
- An acoustic wave source at the inlet boundary
- A barrier with two narrow slits
- Wave propagation and interference patterns in the domain
- Absorbing boundary conditions to prevent reflections

### From Wave Equation to Helmholtz Equation

We start with the **acoustic wave equation** in a homogeneous medium:
$$\frac{\partial^2 p}{\partial t^2} = c^2 \nabla^2 p$$

where:
- $p(x,y,t)$ is the acoustic pressure
- $c$ is the speed of sound
- $t$ is time

For **time-harmonic solutions**, we assume the pressure varies sinusoidally in time:
$$p(x,y,t) = \text{Re}[p(x,y) e^{i\omega t}]$$

where $\omega$ is the angular frequency and $p(x,y)$ is the complex amplitude.

### Why Complex Numbers?

Complex representation is essential because:
1. **Phase information**: The imaginary part captures phase relationships between waves
2. **Superposition**: Complex arithmetic naturally handles wave interference
3. **Boundary conditions**: Absorbing boundaries require complex impedance
4. **Mathematical convenience**: Derivatives become algebraic operations

Substituting the time-harmonic ansatz into the wave equation:
$$\frac{\partial^2}{\partial t^2}[p(x,y) e^{i\omega t}] = c^2 \nabla^2 [p(x,y) e^{i\omega t}]$$

$$-\omega^2 p(x,y) e^{i\omega t} = c^2 \nabla^2 p(x,y) e^{i\omega t}$$

Dividing by $e^{i\omega t}$ and rearranging gives the **Helmholtz equation**:
$$\nabla^2 p + k^2 p = 0$$

where $k = \omega/c$ is the wave number, representing the spatial frequency of the wave.

### Boundary Conditions

The simulation requires different boundary conditions on different parts of the domain boundary:

1. **Inlet boundary ($\Gamma_{in}$)**: Dirichlet condition
   $$p = p_{in} \quad \text{on } \Gamma_{in}$$
   This prescribes the incident wave amplitude and phase.

2. **Wall boundaries ($\Gamma_{wall}$)**: Neumann condition (hard walls)
   $$\frac{\partial p}{\partial n} = 0 \quad \text{on } \Gamma_{wall}$$
   This represents perfect reflection (no normal velocity through solid walls).

3. **Absorbing boundaries ($\Gamma_{abs}$)**: Robin condition
   $$\frac{\partial p}{\partial n} + ik p = 0 \quad \text{on } \Gamma_{abs}$$
   This approximates an infinite domain by absorbing outgoing waves.

These boundary conditions ensure:
- Controlled wave generation at the inlet
- Perfect reflection from solid barriers
- Minimal reflections from artificial boundaries

In [1]:
from mpi4py import MPI
from dolfinx import (
    fem,
    io,
    __version__ as dolfinx_version,
)
from dolfinx.fem.petsc import LinearProblem
import ufl
import numpy as np
import numpy.typing as npt
from packaging.version import Version

# Tags coming from the mesh physical groups
v_bc_wall_tag = 31
v_bc_inlet_tag = 29
v_bc_outlet_tag = 30

mesh_data = io.gmshio.read_from_msh("double_slit_mesh.msh", MPI.COMM_WORLD, 0, gdim=2)
if Version(dolfinx_version) > Version("0.9.0"):
    domain = mesh_data.mesh
    assert mesh_data.facet_tags is not None
    facet_tags = mesh_data.facet_tags
else:
    domain, boundary_tags, facet_tags = mesh_data

Info    : Reading 'double_slit_mesh.msh'...
Info    : 33 entities
Info    : 45295 nodes
Info    : 90590 elements
Info    : Done reading 'double_slit_mesh.msh'


## Mesh and Boundary Conditions

The simulation domain is loaded from a GMSH mesh file (`double_slit_mesh.msh`) that contains:

- **Inlet boundary** (tag 29): Where the acoustic wave is generated
- **Outlet boundary** (tag 30): Where waves exit the domain  
- **Wall boundaries** (tag 31): Solid barriers including the double slit structure

The mesh includes facet tags that identify different boundary regions, allowing us to apply appropriate boundary conditions to each region.

The code handles different versions of DOLFINx to ensure compatibility across versions.

## Function Space and Physical Parameters

We define the function space for our unknown pressure field $p$ and specify the physical parameters for the acoustic wave propagation.

In [2]:

V = fem.functionspace(domain, ("Lagrange", 4))

# Air parameters
omega = 6 * np.pi * 1000  # Angular frequency (rad/s)
rho0 = 1.225  # kg/m^3
c = 340  # m/s
k = omega / c  # Wave number (1/m)
v_n = np.exp(1j * omega * 0)  # Normal velocity amplitude at inlet (m/s)

### Function Space
- **Lagrange elements of order 4**: Higher-order elements provide better accuracy for wave propagation problems
- **Complex-valued**: Required for time-harmonic analysis with phase information

### Physical Parameters
- **Angular frequency (ω)**: 6π × 1000 rad/s ≈ 3 kHz
- **Air density (ρ₀)**: 1.225 kg/m³ (standard air density)
- **Sound speed (c)**: 340 m/s (speed of sound in air)
- **Wave number (k)**: k = ω/c, determines the wavelength
- **Inlet velocity (v_n)**: Complex amplitude of normal velocity at the source

## Integration Measures

We define the integration measure for boundary integrals using UFL (Unified Form Language).

In [3]:
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

The `ds` measure allows us to integrate over specific boundary segments using the facet tags:
- `ds(v_bc_inlet_tag)`: Integration over the inlet boundary
- `ds(v_bc_outlet_tag)`: Integration over the outlet boundary  
- `ds(v_bc_wall_tag)`: Integration over wall boundaries

## Variational Formulation

We formulate the weak form of the Helmholtz equation with appropriate boundary conditions for the double slit problem.

### Mathematical Derivation

Starting from the Helmholtz equation:
$$\nabla^2 p + k^2 p = 0 \quad \text{in } \Omega$$

Multiply by a test function $v$ and integrate over the domain:
$$\int_\Omega (\nabla^2 p + k^2 p) \overline{v} \, d\Omega = 0$$

Using integration by parts on the Laplacian term:
$$\int_\Omega \nabla^2 p \cdot \overline{v} \, d\Omega = -\int_\Omega \nabla p \cdot \nabla \overline{v} \, d\Omega + \int_{\partial\Omega} \frac{\partial p}{\partial n} \overline{v} \, ds$$

This gives the **weak formulation**:
$$-\int_\Omega \nabla p \cdot \nabla \overline{v} \, d\Omega + k^2 \int_\Omega p \overline{v} \, d\Omega + \int_{\partial\Omega} \frac{\partial p}{\partial n} \overline{v} \, ds = 0$$

### Boundary Term Treatment

The boundary integral is handled differently on each boundary:

1. **Wall boundaries**: $\frac{\partial p}{\partial n} = 0$, so the boundary term vanishes.

2. **Absorbing boundaries**: $\frac{\partial p}{\partial n} = -ik p$, giving:
   $$\int_{\Gamma_{abs}} \frac{\partial p}{\partial n} \overline{v} \, ds = -ik \int_{\Gamma_{abs}} p \overline{v} \, ds$$

3. **Inlet boundary**: Dirichlet condition is enforced directly, but we can include a source term:
   $$\int_{\Gamma_{in}} \frac{\partial p}{\partial n} \overline{v} \, ds = -i\omega\rho_0 \int_{\Gamma_{in}} v_n \overline{v} \, ds$$

### Final Variational Form

The complete variational formulation becomes:

**Find** $p \in V$ such that $a(p,v) = L(v)$ for all $v \in V$, where:

$$a(p,v) = \int_\Omega \nabla p \cdot \nabla \overline{v} \, d\Omega - k^2 \int_\Omega p \overline{v} \, d\Omega - ik \int_{\Gamma_{abs}} p \overline{v} \, ds$$

$$L(v) = -i\omega\rho_0 \int_{\Gamma_{in}} v_n \overline{v} \, ds$$

where $V$ is the appropriate function space and $\overline{v}$ denotes the complex conjugate.

In [4]:
p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = (
    ufl.inner(ufl.grad(p), ufl.grad(v)) * ufl.dx
    - (k**2)*p*ufl.conj(v) * ufl.dx
    - 1j*k * p * ufl.conj(v) * ds
    - k**2 * ufl.inner(p, v) * ufl.dx
)
L = -1j * omega * rho0 * ufl.inner(v_n, v) *ds(v_bc_inlet_tag)

## Boundary Conditions and Solution

We apply Dirichlet boundary conditions at the inlet and solve the linear system using PETSc.

In [5]:
bc_inlet = fem.dirichletbc(v_n, fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_inlet_tag)), V)

### Inlet Boundary Condition
- **Dirichlet BC**: Prescribes the pressure amplitude at the inlet
- **DOF location**: Uses topological information to find degrees of freedom on the inlet boundary
- This ensures a controlled wave source at the inlet

In [6]:
p_a = fem.Function(V)
p_a.name = "pressure"

problem = LinearProblem(
    a,
    L,
    u=p_a,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
    bcs=[bc_inlet],
)

### Linear Problem Setup
- **Solution function**: `p_a` stores the computed pressure field
- **PETSc options**: 
  - `"ksp_type": "preonly"`: Direct solver (no iterative method)
  - `"pc_type": "lu"`: LU decomposition for direct factorization
  - MUMPS option for parallel direct solving
- **Boundary conditions**: Applied at the inlet

In [7]:

problem.solve()
p_a.x.scatter_forward()

### Solving the System
1. **solve()**: Assembles and solves the linear system Ax = b
2. **scatter_forward()**: Ensures proper distribution of solution data across MPI processes

The solution `p_a` now contains the complex pressure field showing wave propagation and interference patterns through the double slit.

In [8]:
with io.VTKFile(domain.comm, "solution.pvd", "w") as file:
    file.write_mesh(domain)
    file.write_function([p_a])

## Output and Visualization

The solution is written to a ParaView format file (`solution.pvd`) for visualization:

- **Mesh geometry**: Domain structure including the double slit barrier
- **Pressure field**: Complex-valued solution showing wave amplitude and phase
- **Visualization**: Can be opened in ParaView to see:
  - Wave propagation from inlet
  - Diffraction through the slits  
  - Interference patterns downstream
  - Wave amplitude and phase distributions